This notebook is created at GESIS foe general public to test M3 model for gender inference from images. You can upload your own image or use a url. All data is deleted afterwards.

In [24]:
# #############################################################################
# Importing necessary libraries

# import numpy as np
# import pandas as pd
# import requests as requests
# import json
# from pathlib import Path
# from m3inference import M3Inference
# from collections import OrderedDict
# import pprint
# import urllib.request
# import random
# import re
# import json
# import os
# from PIL import Image
# import csv 
# import sys
# import string 
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#import zipfile as zf


from ipywidgets import TwoByTwoLayout
import ipywidgets as widgets
import requests as requests
from IPython.display import clear_output

In [43]:
def initialize_widgets():
    global upload, link, layout
    upload = widgets.FileUpload(
        accept='image/*', 
        multiple=False  
    )
    link = widgets.Text(
        placeholder='url to an image file',
        description='Link:',
        disabled=False
    )

    layout = TwoByTwoLayout(top_left=upload,
                   top_right=submit,
                   bottom_left=link,
                   bottom_right=clear)
    display(layout)
    
global clear, submit
submit = widgets.Button(description="Submit")
clear = widgets.Button(description="Clear")
initialize_widgets()

def on_clear_clicked(b):
    clear_output()
    initialize_widgets()

def on_submit_clicked(b):
    if upload.value and link.value:
        print('Please choose only one method for input')
        return False
    elif upload.value:
        uploaded_filename = next(iter(upload.value))
        content = upload.value[uploaded_filename]['content']
    elif link.value:
        content = requests.get(link.value).content
    else:
        print('No image has been uploaded')
        return False
    print('An image you submitted:')
    display(widgets.Image(value=content, width=200))

submit.on_click(on_submit_clicked)
clear.on_click(on_clear_clicked)


TwoByTwoLayout(children=(FileUpload(value={}, accept='image/*', description='Upload', layout=Layout(grid_area=…

In [ ]:
# Expanding the zip files

files = zf.ZipFile("/home/sshuser2/IMDB_Images.zip", 'r')
files.extractall('/home/sshuser2/a')
files.close()

# Resize all the images in the same length and breath
def resize_im(image, hight, width):
    Image_a=Image.open(image)# Resizing the image
    resized_im = Image_a.resize((hight, width))
    return resized_im

# Convert all the images in the RGB formate
def Convert_format(resized_image):
    file_name=resized_image
    im = Image.open(resized_image)
    x,y=im.size
    if x!= y:
        #x,y size is not equal then it will resize the images 
        sqrWidth = np.ceil(np.sqrt(im.size[0]*im.size[1])).astype(int)
        im_resize = im.resize((sqrWidth, sqrWidth))
        pix = im_resize.load()
        Image.open(file_name).convert('RGB').save(file_name)
    else:
        Image.open(file_name).convert('RGB').save(file_name)
        
# Generate random string for JSON file
def randomString(stringLength):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

# Generate random numbers
def randomNumber(Numberlength):
    for x in range(Numberlength):
        number= random.randint(1,10001)
    return number

# Running the M3 inference and getting the gender
def Mi_inference(Imagelocation,Imagename):
    config = Path(Imagelocation)
    value=randomNumber(10)
    Dummy_name=randomString(10)

    if config.is_file():                               # here we passing parameters in random both string and number
        data= {
            "description":Dummy_name, 
            "id": value,
            "img_path": Imagelocation, 
            "lang": "en", 
            "name": Dummy_name, 
            "screen_name": Dummy_name
        }
        Json_file='/home/sshuser2/M3/data.jsonl'        # Path to create the Json file
        with open(Json_file, 'w') as json_file:
            json.dump(data, json_file) 
        try:                                            # Here Based on values in output will select the geneder
            m3 = M3Inference()                          # see docstring for details
            pred = m3.infer(Json_file)                  # also see docstring for details
            disc=pprint.pprint(pred)
            main=dict(pred).get(value)
            main_gender=main.get('gender')
            main_gender_female=main_gender.get('female')
            main_gender_male=main_gender.get('male')
            if main_gender_female > main_gender_male :
                Gender='F'
            elif main_gender_female < main_gender_male :
                Gender='M'
            else:
                Gender='N'
        except urllib.error.HTTPError as e:
            print (e)
    else:
        Gender='NI'
    return Gender

# Running the imageson M3 and getting the output
with open('/home/sshuser2/imdb.csv', 'w', newline='') as myfile:             # Here output written in the file
    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
    wr.writerow(['Imagename','G_Gender','M_Gender'])

    for Imagename in os.listdir('/home/sshuser2/a/'):                        # Get the images from directory
        try:
            Image_location="/home/sshuser2/a/"+Imagename                     # Get the image loacation of each image
            resized_image=resize_im(Image_location,200,200)
            Convert_format(Image_location)
            Mi_gender=Mi_inference(Image_location,Imagename)
            wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
            wr.writerow([Imagename,'F',Mi_gender])
        except:
            print("Oops!",sys.exc_info()[0],"occured.")
            print()
